In [1]:
import xarray as xr

In [2]:
calcofi_data = xr.load_dataset(
    "../stations/calcofi/3_post_processed/calcofi_zooplankton.zarr", engine="zarr"
)
calcofi_data

<xarray.Dataset> Size: 346MB
Dimensions:      (is_day: 2, latitude: 55, longitude: 103, time: 7628)
Coordinates:
  * is_day       (is_day) bool 2B False True
  * latitude     (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude    (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
  * time         (time) datetime64[ns] 61kB 1951-01-09 1951-01-10 ... 2023-01-25
Data variables:
    zooplankton  (time, is_day, latitude, longitude) float32 346MB nan ... nan

In [3]:
results = xr.Dataset(
    {
        "day": calcofi_data["zooplankton"].sel(is_day=True).drop_vars("is_day"),
        "night": calcofi_data["zooplankton"].sel(is_day=False).drop_vars("is_day"),
    }
)
results

<xarray.Dataset> Size: 346MB
Dimensions:    (latitude: 55, longitude: 103, time: 7628)
Coordinates:
  * latitude   (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude  (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
  * time       (time) datetime64[ns] 61kB 1951-01-09 1951-01-10 ... 2023-01-25
Data variables:
    day        (time, latitude, longitude) float32 173MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float32 173MB nan nan nan ... nan nan

In [4]:
import plotly.express as px

day_df = (
    results["day"].mean(["latitude", "longitude"]).to_dataframe().reset_index().dropna()
)
night_df = (
    results["night"]
    .mean(["latitude", "longitude"])
    .to_dataframe()
    .reset_index()
    .dropna()
)


fig = px.line(
    title="Day and Night Series", labels={"value": "Abundance", "time": "Time"}
)

fig.add_scatter(x=day_df["time"], y=day_df["day"], mode="lines", name="Day")

fig.add_scatter(x=night_df["time"], y=night_df["night"], mode="lines", name="Night")

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Carbon weight mg/m3",
    legend_title="Day/Night",
    legend=dict(orientation="h"),
)

fig.show()


In [5]:
data_export = xr.Dataset(
    {
        "day": calcofi_data["zooplankton"].sel(is_day=True).drop_vars("is_day"),
        "night": calcofi_data["zooplankton"].sel(is_day=False).drop_vars("is_day"),
    }
)
data_export["day"].attrs = {
    "description": "Zooplankton carbon weight by square meter of sea water at day",
    "units": "mg/m2",
}
data_export["night"].attrs = {
    "description": "Zooplankton carbon weight by square meter of sea water at night",
    "units": "mg/m2",
}
data_export

<xarray.Dataset> Size: 346MB
Dimensions:    (latitude: 55, longitude: 103, time: 7628)
Coordinates:
  * latitude   (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude  (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
  * time       (time) datetime64[ns] 61kB 1951-01-09 1951-01-10 ... 2023-01-25
Data variables:
    day        (time, latitude, longitude) float32 173MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float32 173MB nan nan nan ... nan nan

In [6]:
data_export.to_zarr("Calcofi_zooplankton.zarr", mode="w")